<a href="https://colab.research.google.com/github/lovelyoyrmia/machine-learning-notebook/blob/main/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving ratings.csv to ratings.csv
Saving movies.csv to movies.csv


{'movies.csv': b'movie_id,movie_title,movie_genres_str\n1,Toy Story (1995),"[\'Adventure\', \'Animation\', ""Children\'s""]"\n2,GoldenEye (1995),"[\'unknown\', \'Action\', \'Thriller\']"\n3,Four Rooms (1995),[\'Thriller\']\n4,Get Shorty (1995),"[\'unknown\', ""Children\'s"", \'Documentary\']"\n5,Copycat (1995),"[\'Comedy\', \'Documentary\', \'Thriller\']"\n6,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),[\'Documentary\']\n7,Twelve Monkeys (1995),"[\'Documentary\', \'Sci-Fi\']"\n8,Babe (1995),"[\'Animation\', ""Children\'s"", \'Documentary\']"\n9,Dead Man Walking (1995),[\'Documentary\']\n10,Richard III (1995),"[\'Documentary\', \'Western\']"\n11,Seven (Se7en) (1995),"[\'Comedy\', \'Thriller\']"\n12,"Usual Suspects, The (1995)","[\'Comedy\', \'Thriller\']"\n13,Mighty Aphrodite (1995),"[""Children\'s""]"\n14,"Postino, Il (1994)","[\'Documentary\', \'Romance\']"\n15,Mr. Holland\'s Opus (1995),[\'Documentary\']\n16,French Twist (Gazon maudit) (1995),"[""Children\'s"", \'Romance\']"\

In [ ]:
import tensorflow
import pandas as pd
import sklearn

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
movies.head()

,movie_id,movie_title,movie_genres_str
0,1,Toy Story (1995),"['Adventure', 'Animation', ""Children's""]"
1,2,GoldenEye (1995),"['unknown', 'Action', 'Thriller']"
2,3,Four Rooms (1995),['Thriller']
3,4,Get Shorty (1995),"['unknown', ""Children's"", 'Documentary']"
4,5,Copycat (1995),"['Comedy', 'Documentary', 'Thriller']"


In [ ]:
n_movie = ratings['movie_id'].nunique()
n_user = ratings['user_id'].nunique()

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(ratings, test_size=0.2)

In [ ]:
print(len(train))
print(len(test))

80000
20000


In [ ]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten

# STRUCTURE LAYER
EMBEDDING_DIM = 40

# INPUT LAYER
movie_input = Input(shape=1)
user_input = Input(shape=1)

# EMBEDDING LAYER
movie_embedding = Embedding(n_movie + 1, EMBEDDING_DIM)(movie_input)
user_embedding = Embedding(n_user + 1, EMBEDDING_DIM)(user_input)

movie_flat = Flatten()(movie_embedding)
user_flat = Flatten()(user_embedding)

# OUTPUT LAYER
output = Dot(1)([movie_flat, user_flat])

model = Model([movie_input, user_input], [output])


In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(x=[train['movie_id'], train['user_id']], y=train['user_rating'],
          epochs=30, 
          batch_size=128)

Epoch 1/30
625/625 [==============================] - 2s 3ms/step - loss: 11.9435
Epoch 2/30
625/625 [==============================] - 2s 3ms/step - loss: 2.3568
Epoch 3/30
625/625 [==============================] - 1s 2ms/step - loss: 1.0985
Epoch 4/30
625/625 [==============================] - 2s 2ms/step - loss: 0.9383
Epoch 5/30
625/625 [==============================] - 1s 2ms/step - loss: 0.8878
Epoch 6/30
625/625 [==============================] - 2s 2ms/step - loss: 0.8611
Epoch 7/30
625/625 [==============================] - 2s 2ms/step - loss: 0.8387
Epoch 8/30
625/625 [==============================] - 2s 2ms/step - loss: 0.8184
Epoch 9/30
625/625 [==============================] - 2s 3ms/step - loss: 0.7960
Epoch 10/30
625/625 [==============================] - 1s 2ms/step - loss: 0.7734
Epoch 11/30
625/625 [==============================] - 2s 2ms/step - loss: 0.7497
Epoch 12/30
625/625 [==============================] - 2s 3ms/step - loss: 0.7259
Epoch 13/30
625/625 [===

In [ ]:
model.evaluate(x=[test['movie_id'], test['user_id']], y=test['user_rating'])

625/625 [==============================] - 1s 1ms/step - loss: 0.9691


0.9690820574760437

In [ ]:
model.save('model.h5')

In [ ]:
def get_recommendations(user_id,movies, model):
  movies = movies.copy()
  user_ids = np.array([user_id] * len(movies))
  results = model([movies['movie_id'].values, user_ids]).numpy().reshape(-1)

  movies['predicted_rating'] = pd.Series(results)
  movies = movies.sort_values('predicted_rating', ascending=False)

  print(f'Recommendations for user {user_id}')
  return movies

In [ ]:
get_recommendations(11, movies, model)

Recommendations for user 11


,movie_id,movie_title,movie_genres_str,predicted_rating
314,315,Apt Pupil (1998),"['Documentary', 'Thriller']",5.090863
407,408,"Close Shave, A (1995)","['Adventure', ""Children's"", 'Thriller']",4.921986
514,515,"Boot, Das (1981)","['unknown', 'Documentary', 'Western']",4.873903
1136,1137,Beautiful Thing (1996),"['Documentary', 'Romance']",4.714067
749,750,Amistad (1997),['Documentary'],4.710769
...,...,...,...,...
1542,1543,Johns (1996),['Documentary'],-0.056248
1532,1533,I Don't Want to Talk About It (De eso no se ha...,['Documentary'],-0.058190
1665,1666,Ripe (1996),['Documentary'],-0.063775
1155,1156,Cyclo (1995),"['Comedy', 'Documentary']",-0.099602


In [ ]:
def get_embedding(movies, model):
  vectors = model.layers[2](movies['movie_id'].values).numpy()
  vectors = pd.DataFrame(vectors)
  vectors.to_csv('vectors.tsv', sep='\t', header=False)

  movies.to_csv('meta.tsv', sep='\t')

In [ ]:
get_embedding(movies, model)

In [ ]:

!tensorflowjs_converter-input_format=keras ./model.h5 ./tfjs_model

ERROR: Could not find a version that satisfies the requirement tensorflow.js (from versions: none)
ERROR: No matching distribution found for tensorflow.js
/bin/bash: tensorflowjs_converter-input_format=keras: command not found


In [ ]:
!pip install tensorflowjs
!tensorflowjs_converter --input_format=keras ./model.h5 ./tfjs_model